# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [4]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here
orders = pd.read_csv('Orders.zip', compression='zip', header=0, sep=',')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [8]:
#Method 1 - before Jo´s Tips

#1
table1 = orders.groupby(by = 'CustomerID').sum() [['amount_spent']]

#2
q95 = int(table1.quantile(.95))
q75 = int(table1.quantile(.75))

#3
table1['Label'] = ''
table1.loc[table1['amount_spent'] > q95, 'Label'] = 'VIP'
table1.loc[(table1['amount_spent'] < q75) & (table1['amount_spent'] < q95),'Label'] = 'Preferred'
table1


#Method 2 - I include the country in table 2 so that I can then use in the questions below.

#1
table2 = orders.groupby(by = ['CustomerID','Country']).agg({'amount_spent':'sum'})

#2
q95 = int(table2.quantile(.95))
q75 = int(table2.quantile(.75))

#3
table2['Label'] = ''
table2.loc[table2['amount_spent'] > q95, 'Label'] = 'VIP'
table2.loc[(table2['amount_spent'] < q75) & (table2['amount_spent'] < q95),'Label'] = 'Preferred'
table2

,,amount_spent,Label
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,
12348,Finland,1797.24,
12349,Italy,1757.55,
12350,Norway,334.40,Preferred
...,...,...,...
18280,United Kingdom,180.60,Preferred
18281,United Kingdom,80.82,Preferred
18282,United Kingdom,178.05,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [36]:
# Method 1

table1 = table1.reset_index() #reset index to put Customer ID as column

country = orders.groupby(by = ['CustomerID','Country']).sum() # group orders by CID & Country 
country = country.reset_index() #reset index to put Customer ID & Country as column

#merge Country and Table on Customer ID to get VIP/Preferred cl. per Customer ID)
regroup = (country.merge(table1, how='outer', on=['CustomerID'], sort=False)) [['CustomerID','Country','Label']]

# select columns of VIP Customers & group them by country counting occurences per Country
# sort descending get the Index of Series to get the name of the countries and choose the first entry
regroup[regroup['Label'] == 'VIP'].groupby(by = 'Country').count() ['Label'].sort_values(ascending=False)#.index [0]

# Method 2 - Just group table 2 by Country & Amoun Spent and then I get the entries with 'VIP ' in Label and count them.
# Then sort descending and get the first entry in the index.

table3 = table2.groupby(by = ['Country','amount_spent']).sum()
table3[table3['Label'] == 'VIP'].count(level='Country').sort_values(by='Label',ascending=False).index[0]

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [39]:
#same as previous but changing the condition to get VIP + Preferred class. 

#Method 1
regroup[(regroup['Label'] == 'VIP') | (regroup['Label'] == 'Preferred')].groupby(by = 'Country').count()['Label'].sort_values(ascending=False).index [0]

#Method 2

table3[(table3['Label'] == 'VIP') | (table3['Label'] == 'Preferred')].count(level='Country').sort_values(by='Label',ascending=False).index[0]


'United Kingdom'